In [60]:
!unzip -q 作业-基于OCR的点名检测.zip -d data/

In [61]:
!unzip -q simfang_downyi.com.zip

In [62]:
!unzip -q QQ图片20220520105515.zip -d data/作业-基于OCR的点名检测/0225/

QQ图片20220520105515.png:  mismatching "local" filename (QQхЫ╛чЙЗ20220520105515.png),
         continuing with "central" filename version


In [63]:
!pip install paddleocr

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
You should consider upgrading via the '/opt/conda/envs/python35-paddle120-env/bin/python -m pip install --upgrade pip' command.


In [64]:
import os
import numpy as np
import pandas as pd

In [65]:
word_list = []
datas = []

In [66]:
path = '名单.csv'
data_name = pd.read_csv(path, encoding= 'GBK')

In [67]:
data_name

,序号,姓名
0,1,付相达
1,2,胡可冰
2,3,郭瑞雪
3,4,荘贺翔
4,5,刘营
...,...,...
75,76,莫绍聪
76,77,谢俊杰
77,78,陈德乐
78,79,马珺


In [68]:
for text in data_name['姓名']:
    word_list.append(text)

In [69]:
num_list = []
for num in data_name['序号']:
    num_list.append(num)
num_list

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80]

In [70]:
word_list

['付相达',
 '胡可冰',
 '郭瑞雪',
 '荘贺翔',
 '刘营',
 '张媛媛',
 '王承义',
 '李昶生',
 '赵雨欣',
 '霍玮玮',
 '魏海波',
 '孙润基',
 '侯威辰',
 '刘明鑫',
 '张璐',
 '陆敖',
 '胡安然',
 '郎济诚',
 '张富智',
 '卢泳元',
 '董悦',
 '魏恒通',
 '韩桂兰',
 '杨康',
 '王帅',
 '高智龙',
 '亢清源',
 '毛高翔',
 '黄炎萍',
 '魏宇鑫',
 '田心冉',
 '张世琛',
 '张梦冉',
 '赵豪文',
 '罗嗣颖',
 '黄友高',
 '周鸿',
 '柳江',
 '李彩荣',
 '杨笑豪',
 '郑祥云',
 '吕敏',
 '迟连攀',
 '黄晓雨',
 '吴玉琳',
 '金善玮',
 '郭兆龙',
 '孟庆博',
 '董孝真',
 '李宇轩',
 '徐欣彤',
 '徐欣瑶',
 '宋振永',
 '刘于谦',
 '陈志昂',
 '范文卓',
 '代洪砾',
 '陈云',
 '傅湘婷',
 '毕彦昊',
 '黄浩',
 '李晨',
 '赵家伟',
 '郭雨',
 '曹鸿绪',
 '郭乐彦',
 '迟芮勋',
 '李玮琳',
 '宁豪',
 '李广志',
 '王白茹',
 '潘润',
 '刘帅',
 '冉佳明',
 '陈巧',
 '莫绍聪',
 '谢俊杰',
 '陈德乐',
 '马珺',
 '谭迅捷']

In [71]:
%cd ~
path = 'data/作业-基于OCR的点名检测/'
filelist_dir = os.listdir('data/作业-基于OCR的点名检测')
filelist_dir

/home/aistudio


['0412',
 '0329',
 '0324',
 '0505',
 '0225',
 '0503',
 '0315',
 '0510',
 '0426',
 '0303',
 '作业要求.doc',
 '名单.csv',
 '0407',
 '0405',
 '0421',
 '0419',
 '0322']

In [72]:
filelist_dir.remove('名单.csv')
filelist_dir.remove('作业要求.doc')

In [73]:
filelist_dir.sort()
filelist_dir

['0225',
 '0303',
 '0315',
 '0322',
 '0324',
 '0329',
 '0405',
 '0407',
 '0412',
 '0419',
 '0421',
 '0426',
 '0503',
 '0505',
 '0510']

In [74]:
def getFileList(dir,Filelist, ext=None):
    """
    获取文件夹及其子文件夹中文件列表
    输入 dir：文件夹根目录
    输入 ext: 扩展名
    返回： 文件路径列表
    """
    newDir = dir
    if os.path.isfile(dir):
        if ext is None:
            Filelist.append(dir)
        else:
            if ext in dir[-3:]:
                Filelist.append(dir)
    
    elif os.path.isdir(dir):
        for s in os.listdir(dir):
            newDir=os.path.join(dir,s)
            getFileList(newDir, Filelist, ext)
 
    return Filelist
 
#org_img_folder=path + filelist_dir[0]
 
# 检索文件
#for name in filelist_dir:
#    imglist = getFileList(org_img_folder, [], 'png')
#imglist


In [75]:
chuqin = [[], [], [], [], [], [], [], [], [], [], [], [], [], [], []]
#chuqin = []

In [76]:
from paddleocr import PaddleOCR, draw_ocr
from PIL import Image
# Paddleocr目前支持的多语言语种可以通过修改lang参数进行切换
# 例如`ch`, `en`, `fr`, `german`, `korean`, `japan`
ocr = PaddleOCR(use_angle_cls=True, lang="ch")  # need to run only once to download and load model into memory

i = -1
for name in filelist_dir:
    org_img_folder=path + name
    imglist = getFileList(org_img_folder, [], 'png')
    number = 1
    i = i + 1
    for name_file in imglist:
        img_path = name_file
        result = ocr.ocr(img_path, cls=True)
        for line in result:
            print(line)
        
        # 显示结果

        image = Image.open(img_path).convert('RGB')
        boxes = [line[0] for line in result]
        txts = [line[1][0] for line in result]
        print(i)
        for word_x in txts:
            if word_x == '马':
                word_x = '马珺'
            if word_x == '潘润家长':
                word_x = '潘润'
            for word_y in word_list:
                if word_x[-3:] == word_y or word_x[-2:] == word_y[-2:]:
                    flag = 0
                    for word_z in chuqin[i]:
                        if word_y == word_z:
                            flag = 1
                    if flag == 0:
                        chuqin[i].append(word_y)
        scores = [line[1][1] for line in result]
        im_show = draw_ocr(image, boxes, txts, scores, font_path='simfang_ttf/simfang.ttf')
        im_show = Image.fromarray(im_show)
        im_show.save('result/result' + name + '_' + str(number) + '.jpg')
        number = number + 1

In [77]:
chuqin[0]

['胡可冰',
 '胡安然',
 '侯威辰',
 '张梦冉',
 '莫绍聪',
 '吴玉琳',
 '刘于谦',
 '迟芮勋',
 '谢俊杰',
 '王帅',
 '赵豪文',
 '陈巧',
 '刘帅',
 '马珺',
 '宁豪',
 '徐欣彤',
 '徐欣瑶',
 '潘润',
 '田心冉',
 '魏海波',
 '张世琛',
 '高智龙',
 '王白茹',
 '黄友高',
 '李昶生',
 '刘明鑫',
 '陆敖',
 '黄晓雨',
 '霍玮玮',
 '郎济诚',
 '李彩荣',
 '卢泳元',
 '孙润基',
 '王承义',
 '张媛媛',
 '赵家伟',
 '李晨',
 '李玮琳',
 '刘营',
 '罗嗣颖',
 '魏宇鑫',
 '宋振永',
 '张富智',
 '周鸿',
 '荘贺翔',
 '毕彦昊',
 '陈云',
 '陈志昂',
 '迟连攀',
 '董孝真',
 '傅湘婷',
 '李宇轩',
 '吕敏',
 '杨康',
 '郑祥云',
 '黄浩',
 '郭瑞雪',
 '曹鸿绪',
 '赵雨欣',
 '亢清源',
 '黄炎萍',
 '郭兆龙',
 '郭雨',
 '冉佳明',
 '代洪砾',
 '郭乐彦',
 '韩桂兰',
 '柳江',
 '毛高翔']

In [78]:
for i in range(15):
    print(len(chuqin[i]))

69
74
73
75
75
79
69
75
77
79
78
71
73
78
76


In [79]:
tongji = np.zeros((80, 15))
for i in range(15):
    for j in range(80):
        for word_exam in chuqin[i]:
            if word_list[j] == word_exam:
                tongji[j][i] = 1

In [80]:
columns = ['0']*17
columns[0] = 'number'
columns[1] = 'name'
k = 2
for filename_x in filelist_dir:
    columns[k] = filename_x
    k = k + 1
data = [['0']*17 for i in range(80)]
for i in range(80):
    data[i][0] = num_list[i]
    data[i][1] = word_list[i]
for i in range(2, 17):
    for j in range(80):
        data[j][i] = tongji[j][i - 2]
df = pd.DataFrame(columns=columns, data=data)

df

,number,name,0225,0303,0315,0322,0324,0329,0405,0407,0412,0419,0421,0426,0503,0505,0510
0,1,付相达,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,2,胡可冰,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,3,郭瑞雪,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,4,荘贺翔,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0
4,5,刘营,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,76,莫绍聪,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
76,77,谢俊杰,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
77,78,陈德乐,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
78,79,马珺,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [81]:
#将DataFrame存储为csv,index表示是否显示行名，default=True，path指写入的文件名称
df.to_csv('出勤统计.csv', index=True, sep=',', encoding = 'utf_8_sig')